In [1]:
from secret_key import openai_api_key

In [2]:
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", api_key=openai_api_key)

In [3]:
llm.invoke("How far is the moon away from earth in km ?")

"\n\nThe average distance between the moon and Earth is approximately 384,400 kilometers. However, the distance can vary between 363,300 kilometers (perigee) and 405,500 kilometers (apogee) due to the moon's elliptical orbit."

In [27]:
llm.invoke("Who is the current President of Japan and what is his age ?")

'\n\nThe current President of Japan is Yoshihide Suga who is 72 years old (as of 2021).'

In [5]:
llm.invoke("What are the hobbies of Milton ?")

'\n\nThere is no specific information available about the hobbies of Milton, as it can refer to various individuals with the same name. However, some possible hobbies that people named Milton may have could include reading, writing, playing sports, gardening, cooking, painting, photography, hiking, or playing musical instruments.'

In [6]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo-0125", api_key=openai_api_key)

In [70]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are an expert in Python programming language"),
    HumanMessage(content="Tell me about scikit learn in 2 sentences"),
]

In [71]:
chat.invoke(messages)

AIMessage(content='Scikit-learn is a popular machine learning library in Python that provides efficient tools for data analysis and modeling. It includes various algorithms for classification, regression, clustering, dimensionality reduction, and more, making it a versatile choice for machine learning tasks.', response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 29, 'total_tokens': 79}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-acb15958-7570-4d3d-8e2f-a5c0a49e5527-0')

In [7]:
from langchain import PromptTemplate
template = "I love {food_item}.  Give me a short overview of the item in 1 sentence ?"

In [8]:
prompt = PromptTemplate(input_variables=["food_item"], template=template)
final_prompt = prompt.format(food_item='Biryani')

In [9]:
llm.invoke(final_prompt)

'\n\nBiryani is a fragrant and flavorful Indian rice dish typically made with spiced meat or vegetables and basmati rice, cooked together in layers for a delicious and satisfying meal.'

In [10]:
from langchain.chains import LLMChain, SimpleSequentialChain
template = "What is the capital of {country} in one word"
prompt_template = PromptTemplate(input_variables=["country"], template=template)

country_chain = LLMChain(llm=llm, prompt=prompt_template)

In [11]:
template = "Provide a sentence about the {city}"
prompt_template = PromptTemplate(input_variables=["city"], template=template)

city_chain = LLMChain(llm=llm, prompt=prompt_template)

In [12]:
final_chain = SimpleSequentialChain(chains=[country_chain, city_chain], verbose=True)
final_chain.invoke("India")



> Entering new SimpleSequentialChain chain...


Delhi
 is the capital city of India, known for its bustling markets, historic monuments, and diverse culture.

> Finished chain.


{'input': 'India',
 'output': ' is the capital city of India, known for its bustling markets, historic monuments, and diverse culture.'}

In [13]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [14]:
query = "This is my first attempt at converting a sentence to array of numbers"

In [15]:
embedded_query = embeddings_model.embed_query(query)
embedded_query[:5]

[-0.011082770901458278,
 0.012362078392377806,
 0.0009073163692367308,
 -0.010152365707513862,
 0.0002659522790238687]

In [16]:
print (f"Embedded query length: {len(embedded_query)}")

Embedded query length: 1536


In [17]:
from langchain.document_loaders import TextLoader
my_personal_data_document = TextLoader('personaldata.txt').load()

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=20)

In [19]:
my_personal_data_split_documents = text_splitter.split_documents(my_personal_data_document)
print (f"Split document length: {len(my_personal_data_split_documents)}")

Split document length: 32


In [20]:
for index, _ in enumerate(my_personal_data_split_documents):
    print(f"Split {index}, size: {my_personal_data_split_documents[index]}")

Split 0, size: page_content='Name: Milton Chandradas\nAge: 50\nOccupation: Software Industry Professional\nCountry: USA\n\nHobbies:' metadata={'source': 'personaldata.txt'}
Split 1, size: page_content='Photography: Milton enjoys capturing moments in nature and urban landscapes. He often goes on photography expeditions during weekends.' metadata={'source': 'personaldata.txt'}
Split 2, size: page_content='Cooking: Milton finds solace in experimenting with new recipes in his kitchen. He loves hosting dinner parties for friends and family.' metadata={'source': 'personaldata.txt'}
Split 3, size: page_content='Reading: Milton is an avid reader, particularly interested in science fiction and historical non-fiction. He dedicates some time each day to dive' metadata={'source': 'personaldata.txt'}
Split 4, size: page_content='each day to dive into a good book.' metadata={'source': 'personaldata.txt'}
Split 5, size: page_content='Work:' metadata={'source': 'personaldata.txt'}
Split 6, size: page_

In [21]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(my_personal_data_split_documents, embeddings_model)

In [22]:
query = "What are the hobbies of Milton"
docs = db.similarity_search(query, k=4)

In [24]:
print(docs[1].page_content)

Photography: Milton enjoys capturing moments in nature and urban landscapes. He often goes on photography expeditions during weekends.


In [25]:
context = "\n\n".join([doc.page_content for doc in docs])
prompt = f"Context:\n{context}\n\nQuestion: {query}"

In [26]:
llm.invoke(prompt)


' Chandradas?\n\nAnswer: Milton Chandradas enjoys cooking, photography, and occasionally struggling with work-life balance. '

In [28]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from secret_key import serpapi_api_key


In [29]:
toolkit = load_tools(["serpapi", "llm-math"], llm=llm, serpapi_api_key=serpapi_api_key)

In [30]:
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)

In [31]:
response = agent({"input":"Who is the current President of Japan and what is his age ?"})



> Entering new AgentExecutor chain...
 I should use a search engine to find this information.
Action: Search
Action Input: "current President of Japan"
Observation: {'type': 'organic_result', 'title': 'Fumio Kishida', 'source': 'Wikipedia'}
Thought: Now I know the name of the current President of Japan.
Action: Search
Action Input: "age of Fumio Kishida"
Observation: 66 years
Thought: I now know the final answer.
Final Answer: Fumio Kishida is 66 years old.

> Finished chain.


In [33]:
response = agent({"input":"What is the age difference between current President of Japan and Shahrukh Khan ?"})



> Entering new AgentExecutor chain...
 I should use a search engine to find information about the age of the current President of Japan and Shahrukh Khan.
Action: Search
Action Input: "age difference between current President of Japan and Shahrukh Khan"
Observation: ["Sharukh Khan's age difference between his romantic interests over the last 10 years ; Deepika Padukone, 21 years ; Kajol, 10 years ; Mahira, 20 ...", '"Zero," released in 2018, featured Shah Rukh Khan alongside Anushka Sharma, with a significant age difference of 22 years. This wasn\'t their ...', 'Shah Nawaz Khan (general) ; (1914-01-24)24 January 1914. Matore, Punjab, British India (present-day Punjab, Pakistan) · 9 December 1983(1983-12-09) (aged 69)', 'MUMBAI — Shah Rukh Khan has many titles. The 48-year-old Bollywood superstar is known as King Khan, King of Bollywood and Bollywood Badshah ...', 'National Creator Award Winner Travel Creator 2024 ✈️ Cultural Brand Ambassador, India Founder @curly.tales @curlytalesme 